# Transfer Learning with Tensorflow Part 1 : Feature Extraction

Transfer learning is leveraging a working models exisiting architecture and learned patterns

Benifits
- Can leverage an exsting NN architecture proven to work
- Can leverage a working NN which has already learned the patterns and we can adapt the patterns our problem

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Downloading and becomming one with the data

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2021-03-26 04:34:41--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 173.194.217.128, 173.194.218.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   173MB/s    in 0.9s    

2021-03-26 04:34:42 (173 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [ ]:
!unzip 10_food_classes_10_percent.zip

Archive:  10_food_classes_10_percent.zip
replace __MACOSX/._10_food_classes_10_percent? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os

In [ ]:
# How many images in each folder ? 
# Walk through 10% data dir and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/tes

## Create data loaders (preparing the data)

We'll use the ImageDataGenerator class to load our images in batches

In [ ]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"
train_datagen = ImageDataGenerator(rescale=1/255.0)
test_datagen = ImageDataGenerator(rescale=1/255.0)
print('Training Images')
train_data_10_percent = train_datagen.flow_from_directory(train_dir,target_size=IMAGE_SHAPE)
print('Testing Images')
test_data_10_percent = test_datagen.flow_from_directory(test_dir,target_size=IMAGE_SHAPE)

Training Images
Found 750 images belonging to 10 classes.
Testing Images
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run while our model trains)

Callbacks are extra funtionality you can add to your model to be performed during or after training. Some of the most popular callbacks:

* Tracking experiments (Tensorboard)
* Model check point (ModelCheckpoint)
* Stop overfitting (EarlyStopping)

In [ ]:
import tensorflow as tf

In [ ]:

# Create TensorBoard callback (functionized beause we need to create a new one for each model)
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

## Creating models using tensorflow hub

So in the past we have used tensorflow to make models from sctrach but now we are going to do a same like proccess but the architecture is coming from https://tfhub.dev/

Browsing the Tensorflow hub page we found the following model (feature vector)
https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1

In [ ]:
# Lets compare the following 2 models
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
efficentnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Lets make a create_model funtion to create a model from a url
def create_model(model_url,num_classes=10):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                          trainable=False, # remove all of the trained patterns
                                          name="feature_extraction_layer",
                                          input_shape=(224,224,3)
                                           )
  model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes,activation='softmax')
  ])
  return model

In [ ]:
# Create and testing resnet tensorflow hub feature extraction model
resnet_model = create_model(resnet_url,num_classes=train_data_10_percent.num_classes)

In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_extraction_layer (Ke (None, 2048)              23564800  
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
# Compiel our resnet moel
resnet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
resnet_model.fit(train_data_10_percent,epochs=5,callbacks=[create_tensorboard_callback('tensorflow_hub','resnet50v2')],validation_data=test_data_10_percent,)

Saving TensorBoard log files to: tensorflow_hub/resnet50v2/20210326-071015
Epoch 1/5
24/24 [==============================] - 435s 19s/step - loss: 2.2868 - accuracy: 0.2372 - val_loss: 1.2030 - val_accuracy: 0.6124
Epoch 2/5
24/24 [==============================] - 432s 19s/step - loss: 1.0109 - accuracy: 0.7096 - val_loss: 0.8469 - val_accuracy: 0.7396
Epoch 3/5
24/24 [==============================] - 434s 19s/step - loss: 0.6083 - accuracy: 0.8281 - val_loss: 0.7434 - val_accuracy: 0.7644
Epoch 4/5
24/24 [==============================] - 430s 19s/step - loss: 0.4686 - accuracy: 0.8672 - val_loss: 0.7003 - val_accuracy: 0.7772
Epoch 5/5
24/24 [==============================] - 427s 18s/step - loss: 0.4231 - accuracy: 0.8962 - val_loss: 0.6723 - val_accuracy: 0.7848


Wow ! 

That is incrediable. Our transfer learning feature extractor model out performs ALL of the previous models we built by hand. and with a quicker time and with only 10% of the training examples

In [ ]:
import pandas as pd
preds = pd.DataFrame(resnet_model.evaluate(test_data_10_percent))

79/79 [==============================] - 330s 4s/step - loss: 0.6723 - accuracy: 0.7848


In [ ]:
preds

,0
0,0.672322
1,0.784800


## Creating and testing EfficentNetB0 Tensorflow hub feature extraction model

In [ ]:
efficent_model = create_model(efficentnet_url)
efficent_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
efficent_history = efficent_model.fit(train_data_10_percent,epochs=5,callbacks=[create_tensorboard_callback('tensorflow_hub','efficentnetb0')],validation_data=test_data_10_percent,)

Saving TensorBoard log files to: tensorflow_hub/efficentnetb0/20210326-075619
Epoch 1/5
24/24 [==============================] - 221s 9s/step - loss: 2.0921 - accuracy: 0.2906 - val_loss: 1.3130 - val_accuracy: 0.7360
Epoch 2/5
24/24 [==============================] - 208s 9s/step - loss: 1.1645 - accuracy: 0.7476 - val_loss: 0.8707 - val_accuracy: 0.8156
Epoch 3/5
24/24 [==============================] - 206s 9s/step - loss: 0.7697 - accuracy: 0.8586 - val_loss: 0.7001 - val_accuracy: 0.8408
Epoch 4/5
24/24 [==============================] - 205s 9s/step - loss: 0.6286 - accuracy: 0.8766 - val_loss: 0.6144 - val_accuracy: 0.8540
Epoch 5/5
24/24 [==============================] - 206s 9s/step - loss: 0.5226 - accuracy: 0.9100 - val_loss: 0.5584 - val_accuracy: 0.8612


In [ ]:
pd.DataFrame(efficent_model.evaluate(test_data_10_percent))

In [ ]:
efficent_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_extraction_layer (Ke (None, 1280)              4049564   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                12810     
Total params: 4,062,374
Trainable params: 12,810
Non-trainable params: 4,049,564
_________________________________________________________________


In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_extraction_layer (Ke (None, 2048)              23564800  
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


## Different types of transger learning

* "AS if" transfer learning - using an existing model with no change what so ever
* "Feature Extraction" transfer learning - us the pre learned pattern of an existing model
* "Find tuning" transfer learning - use the pretrane model and fine tune the underlying layers

## Comparing our models results using Tensorboard

> Warning : When you upload things to tensorboard.dev your experiments are public so if you are running private experiments (things you dont want others to see) dont use tensorboard !!!!

In [ ]:

# Upload TensorBoard dev records
!tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "EfficientNetB0 vs. ResNet50V2" \
  --description "Comparing two different TF Hub feature extraction model architectures using 10% of the training data" \
  --one_shot

In [ ]:
# Link https://tensorboard.dev/experiment/dQBrpdwIRgS2qI0Andv8Yg/#scalars

In [ ]:
!tensorboard dev list

In [ ]:
# Delete
# !tensorboard dev delete --experiment_id 

In [ ]:
!tensorboard dev list